In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import StringIO
import regex as re

# WILDFIRE-API

In [2]:
MAP_KEY = "85ad147fd1890d06e0e2db5bb9d95b57"

da_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_SP/USA/1/2010-1-02'
df = pd.read_csv(da_url)
df.head()

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,USA,19.5339,-155.8204,341.8,1.3,1.1,2010-01-02,3,Aqua,MODIS,91,6.03,308.3,55.2,D,0
1,USA,29.8817,-91.9583,300.2,1.2,1.1,2010-01-02,432,Terra,MODIS,19,6.03,274.1,11.4,N,0
2,USA,32.0648,-97.4609,334.4,1.1,1.0,2010-01-02,433,Terra,MODIS,100,6.03,274.6,43.2,N,0
3,USA,39.2161,-76.4721,312.0,1.0,1.0,2010-01-02,711,Aqua,MODIS,84,6.03,270.4,16.7,N,2
4,USA,33.9451,-79.6548,300.7,1.0,1.0,2010-01-02,713,Aqua,MODIS,33,6.03,272.9,9.7,N,0


In [3]:
after_rounding = df.copy()
after_rounding['latitude'] = np.round(df['latitude'])
after_rounding['longitude']  = np.round(df['longitude'])
after_rounding.head()


,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,USA,20.0,-156.0,341.8,1.3,1.1,2010-01-02,3,Aqua,MODIS,91,6.03,308.3,55.2,D,0
1,USA,30.0,-92.0,300.2,1.2,1.1,2010-01-02,432,Terra,MODIS,19,6.03,274.1,11.4,N,0
2,USA,32.0,-97.0,334.4,1.1,1.0,2010-01-02,433,Terra,MODIS,100,6.03,274.6,43.2,N,0
3,USA,39.0,-76.0,312.0,1.0,1.0,2010-01-02,711,Aqua,MODIS,84,6.03,270.4,16.7,N,2
4,USA,34.0,-80.0,300.7,1.0,1.0,2010-01-02,713,Aqua,MODIS,33,6.03,272.9,9.7,N,0


In [4]:
fire_lat = after_rounding['latitude'].array
fire_long = after_rounding['longitude'].array
fire_lat

<NumpyExtensionArray>
[20.0, 30.0, 32.0, 39.0, 34.0, 27.0, 27.0, 27.0, 27.0, 34.0, 31.0, 31.0, 32.0,
 32.0, 33.0, 33.0, 34.0]
Length: 17, dtype: float64

# WEATHER-API

In [5]:
historical_data_url = "https://historical-forecast-api.open-meteo.com/v1/forecast"

param = {
    "latitude": fire_lat,
    "longitude": fire_long,
    "daily": ["temperature_2m_max", "temperature_2m_min",
              "precipitation_sum", "rain_sum", "showers_sum",
              "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max",
              "et0_fao_evapotranspiration"],
    "start_date": "2024-10-21",
    "end_date": "2025-01-19",
}

data = requests.get(historical_data_url, params=param);
clean = re.split(r'(?<=\}),(?=\{)', data.text)
cleaned_df = pd.DataFrame()
for coordinate_json in clean:
    extract_cords = re.search(r'{"latitude":[\d\.-]+,"longitude":[\d\.-]+,', coordinate_json)
    extract_output = re.search(r'"daily":{(.*})}', coordinate_json)
    cleaned_df = pd.concat([cleaned_df, pd.read_json(StringIO(extract_cords.group() + extract_output.group(1)))])
divide_time = cleaned_df['time'].str.split(r'\-')
cleaned_df['year'] = divide_time.str[0]
cleaned_df['month'] = divide_time.str[1]
cleaned_df['day'] = divide_time.str[2]
cleaned_df.drop(columns=['time'], inplace=True)
new_column_order = ['latitude', 'longitude', 'year', 'month', 'day',
                    'temperature_2m_max', 'temperature_2m_min',
                    'precipitation_sum', 'rain_sum', 'showers_sum',
                    'snowfall_sum', 'wind_speed_10m_max', 'wind_gusts_10m_max',
                    'et0_fao_evapotranspiration']
cleaned_df = cleaned_df[new_column_order]

In [6]:
cleaned_df

,latitude,longitude,year,month,day,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,wind_speed_10m_max,wind_gusts_10m_max,et0_fao_evapotranspiration
0,20.1250,-155.87500,2024,10,21,30.6,24.3,0.0,0.0,0.0,0.0,9.6,30.2,4.38
1,20.1250,-155.87500,2024,10,22,28.9,24.1,0.0,0.0,0.0,0.0,11.2,21.2,3.70
2,20.1250,-155.87500,2024,10,23,29.5,25.2,0.0,0.0,0.0,0.0,12.9,23.8,3.50
3,20.1250,-155.87500,2024,10,24,30.7,25.2,0.1,0.0,0.1,0.0,10.0,30.2,4.12
4,20.1250,-155.87500,2024,10,25,29.7,24.9,0.0,0.0,0.0,0.0,7.6,25.6,4.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,34.0115,-105.01024,2025,01,15,10.5,-4.9,0.0,0.0,0.0,0.0,27.5,37.4,2.70
87,34.0115,-105.01024,2025,01,16,14.2,-3.8,0.0,0.0,0.0,0.0,20.5,29.2,2.96
88,34.0115,-105.01024,2025,01,17,12.6,1.9,0.0,0.0,0.0,0.0,60.8,86.4,3.37
89,34.0115,-105.01024,2025,01,18,12.1,-2.4,0.0,0.0,0.0,0.0,43.6,66.6,3.48
